<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**NERO Winter School training**</font> <br>
**Copyright:** (c) 2025 EUMETSAT <br>
**License:** GPL-3.0-or-later <br>
**Authors:** Dominika Leskow-Czyżewska (EUMETSAT), based on <a href='https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store/-/blob/master/2_1_Customising_products.ipynb?ref_type=heads'>EUMETSAT Data Services training</a>

<hr>

## Sentinel-3 data with the EUMETSAT Data Tailor and EUMDAC

### Data used

| Product Description  | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-----------------:|
| Sentinel-3 OLCI level 1B full resolution  | EO:EUM:DAT:0409 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0409?query=olci&s=extended) |
| Sentinel-3 SLSTR Level 1B Radiances and Brightness Temperatures | EO:EUM:DAT:0411 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0411?query=slstr&s=extended) |


### What will this module teach you?

This module will show you how to:<br>
1. Select a product from the <b>Data Store</b> via EUMDAC library
2. Pass the product to the <b>Data Tailor</b> via EUMDAC library to remotely customise products prior to download
3. Retrieve your customised products

### Use EUMDAC library

In this section we will demonstrate how you can use the Data Tailor web service to customise products you select from the Data Store. This customisation is done <b><u>prior</u></b> to downloading the products, so it is useful for users who wish to refine their data stream. We can do this by using EUMDAC, an EUMETSAT Python library to handle requests and responses of the APIs.

You will find an installation guide, and further information about the usage of EUMDAC, here: https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide

### Authentication

After installing the eumdac we are calling the Token. It takes care of requesting a new value after expiration.

In [1]:
# Import the library EUMDAC
import eumdac
import time
import requests
import yaml
import os

In [2]:
run_name = "varnavas_example"
output_dir = './example_data/'

In [3]:
import credentials

In [4]:
# Insert your personal key and secret into the single quotes

consumer_key=credentials.EUMDAC_CONSUMER_KEY
consumer_secret=credentials.EUMDAC_CONSUMER_SECRET

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

In [5]:
try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as exc:
    print(f"Error when trying the request to the server: '{exc}'")

This token 'b26f1175-c0f3-3bcc-b8e4-36ecce27840a' expires 2025-02-19 23:03:29.175159


### Selecting products from the Data Store

For this demonstration, we are going to select a Sentinel-3 L1B OLCI product (collection ID: EO:EUM:DAT:0409). Later on, feel free to comment and uncomment one of the sections (select the lines and click Ctrl+/).

In [6]:
folder_name = 'S3_OLCI'
collectionID = 'EO:EUM:DAT:0409' #OLCI
chain_path = 'input_graphs/olci_subset.yaml'

# folder_name = 'S3_SLSTR_SOLAR'
# collectionID = 'EO:EUM:DAT:0411' #SLSTR
# chain_path = 'input_graphs/slstr_solar_subset.yaml'

# folder_name = 'S3_SLSTR_THERMAL'
# collectionID = 'EO:EUM:DAT:0411' #SLSTR
# chain_path = 'input_graphs/slstr_thermal_subset.yaml'

In [7]:
# Create a download directory for our downloaded products
download_dir = os.path.join(output_dir, run_name, 'Satellite_Imagery', folder_name)
os.makedirs(download_dir, exist_ok=True)

In [8]:
datastore = eumdac.DataStore(token)

# Selecting the collection and product.
selected_collection = datastore.get_collection(collectionID)

# Set sensing start and end time
start_time = "2024-08-11T00:00:00"
end_time = "2024-08-13T23:59:59"

#  lat-lon geographical bounds of search area
W = 23.3
S = 37.8
E = 24.5
N = 38.5
bounding_box = f'{W}, {S}, {E}, {N}'  # West, South, East, North

selected_products = selected_collection.search(
    bbox=bounding_box,
    dtstart=start_time,
    dtend=end_time)

print(f'Found Datasets: {selected_products.total_results} datasets for the given time range and geographical area')
for product in selected_products:
    print(str(product))

Found Datasets: 3 datasets for the given time range and geographical area
S3A_OL_1_EFR____20240813T082011_20240813T082311_20240814T104955_0179_115_349_2340_MAR_O_NT_004.SEN3
S3B_OL_1_EFR____20240812T080738_20240812T081038_20240813T112859_0179_096_192_2340_MAR_O_NT_004.SEN3
S3B_OL_1_EFR____20240811T083349_20240811T083649_20240812T120918_0179_096_178_2340_MAR_O_NT_004.SEN3


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Find more information about EUMDAC errors, their causes and possible solutions, in our knowledge base: <a href="https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling">https://user.eumetsat.int/resources/user-guides/eumetsat-data-access-client-eumdac-guide#ID-Exception-handling</a>
</div>

## Customising products with the Data Tailor

To customise a product with the Data Tailor, we need to provide following information;
* A product object
* A chain configuration

We already have the product object, 'latest'. Now, we will define our chain configuration.

In [9]:
datatailor = eumdac.DataTailor(token)

# To check if Data Tailor works as expected, we are requesting our quota information
try:
    display(datatailor.quota)
except eumdac.datatailor.DataTailorError as error:
    print(f"Error related to the Data Tailor: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

{'total': 1,
 'data': {'dominikalc': {'disk_quota_active': True,
   'user_quota': 20000.0,
   'space_usage_percentage': 0.0,
   'space_usage': 0.094362,
   'workspace_dir_size': 0.0,
   'log_dir_size': 0.0,
   'output_dir_size': 0.0,
   'nr_customisations': 0,
   'unit_of_size': 'MB'}}}

We have pre-prepared a few tailoring chains which might be useful. We load one of them with our desired customisation operations.

In [10]:
# Creating custom tailoring chain.
import yaml

# Read the YAML file
with open(chain_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

# Create the Chain object by unpacking all YAML content
chain = eumdac.tailor_models.Chain(**yaml_data)

## Submitting a customisation and downloading the products

Now we can submit our customisation. It is possible to submit up to 3 customisations up the same time, therefore we specify `MAX_QUOTA`.

In [11]:
import fnmatch
import shutil
from concurrent.futures import ThreadPoolExecutor

MAX_QUOTA = 3

We define a function for a single customisation. We define a time duration for the interval with which to query the status of the customisation. Once the customisation has finished, we download the result.

In [12]:
def exec_data_tailor_process(product):
    print(f"Starting Data Tailor for product at {product}")
    # Actual customisation submission
    customisation = datatailor.new_customisation(product, chain=chain)

    # Printing customisation status
    sleep_time = 10 # seconds
    while True:
        status = customisation.status
        if "DONE" in status:
            print(f"Customisation {customisation._id} is successfully completed.")
            print(f"Downloading the output of the customisation {customisation._id}")
            cust_files = fnmatch.filter(customisation.outputs, '*')[0]
            with customisation.stream_output(cust_files) as stream, open(os.path.join(download_dir, stream.name), mode='wb') as fdst:
                shutil.copyfileobj(stream, fdst)
                file_path = os.path.join(download_dir, stream.name)
            print(f"Download finished for customisation {customisation._id}.")
            break
        elif status in ["ERROR", "FAILED", "DELETED", "KILLED", "INACTIVE"]:
            print(f"Customisation {customisation._id} was unsuccessful. Customisation log is printed.\n")
            print(customisation.logfile)
            break
        elif "QUEUED" in status:
            print(f"Customisation {customisation._id} is queued.")
        elif "RUNNING" in status:
            print(f"Customisation {customisation._id} is running.")
        time.sleep(sleep_time)

    print(f"Finishing Data Tailor for product at {product}")
    return file_path
    

We submit customisations for all the found products. 

In [13]:
with ThreadPoolExecutor(max_workers=MAX_QUOTA) as executor:
    futures = [executor.submit(exec_data_tailor_process, product) for product in selected_products]

    # Get the results as they complete
    for future in futures:
        print(future.result())

Starting Data Tailor for product at S3A_OL_1_EFR____20240813T082011_20240813T082311_20240814T104955_0179_115_349_2340_MAR_O_NT_004.SEN3
Starting Data Tailor for product at S3B_OL_1_EFR____20240812T080738_20240812T081038_20240813T112859_0179_096_192_2340_MAR_O_NT_004.SEN3
Starting Data Tailor for product at S3B_OL_1_EFR____20240811T083349_20240811T083649_20240812T120918_0179_096_178_2340_MAR_O_NT_004.SEN3
Customisation 89061cda is running.
Customisation 4c6b4cf7 is running.
Customisation abcd9ed3 is running.
Customisation 4c6b4cf7 is running.
Customisation abcd9ed3 is running.
Customisation 89061cda is running.
Customisation abcd9ed3 is running.
Customisation 4c6b4cf7 is running.
Customisation 89061cda is running.
Customisation 4c6b4cf7 is running.
Customisation abcd9ed3 is running.
Customisation 89061cda is running.
Customisation 4c6b4cf7 is running.
Customisation abcd9ed3 is running.
Customisation 89061cda is running.
Customisation 4c6b4cf7 is running.
Customisation abcd9ed3 is runnin

In [14]:
result_paths = []
for future in futures:
    result_paths.append(future.result())

result_paths

['./example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240813T082029Z_20240813T082329Z_epct_4c6b4cf7_FP.tif',
 './example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240812T080756Z_20240812T081056Z_epct_abcd9ed3_FP.tif',
 './example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240811T083407Z_20240811T083707Z_epct_89061cda_FP.tif']

### Clearing customisations from the Data Tailor

The Data Tailor Web Service has a 20 Gb limit, so it's **important** to clear old customisations. First, we check our quota:

In [15]:
try:
    display(datatailor.quota)
except eumdac.datatailor.DataTailorError as error:
    print(f"Error related to the Data Tailor: '{error.msg}'")
except requests.exceptions.RequestException as error:
    print(f"Unexpected error: {error}")

{'total': 1,
 'data': {'dominikalc': {'disk_quota_active': True,
   'user_quota': 20000.0,
   'space_usage_percentage': 2.53,
   'space_usage': 505.913626,
   'workspace_dir_size': 0.0,
   'log_dir_size': 0.01226,
   'output_dir_size': 505.807004,
   'nr_customisations': 3,
   'unit_of_size': 'MB'}}}

To delete all the customisations we have just created, simply call the delete function:

In [16]:
# Clearing all customisations from the Data Tailor

for customisation in datatailor.customisations:
    if customisation.status in ['QUEUED', 'INACTIVE', 'RUNNING']:
        customisation.kill()
        print(f'Delete {customisation.status} customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except Exception as error:
            print("Unexpected error:", error)
    else:
        print(f'Delete completed customisation {customisation} from {customisation.creation_time} UTC.')
        try:
            customisation.delete()
        except eumdac.datatailor.CustomisationError as error:
            print("Customisation Error:", error)
        except requests.exceptions.RequestException as error:
            print("Unexpected error:", error)

Delete completed customisation 4c6b4cf7 from 2025-02-19 17:55:41 UTC.
Delete completed customisation abcd9ed3 from 2025-02-19 17:55:41 UTC.
Delete completed customisation 89061cda from 2025-02-19 17:55:41 UTC.


### Cropping the data

At the moment, the Data Tailor doesn't have a capability to crop the Sentinel-3 Level 1B data. Therefore, we are going to do it here.

In [17]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import os
from pathlib import Path

In [18]:
# Define bounding box (xmin, ymin, xmax, ymax)
bounding_box = [W, S, E, N]

for input_tiff in result_paths:
    # Input GeoTIFF path
    #input_tiff = result_paths[1]
    temp_tiff = Path(download_dir) / "temp_subset.tif"

    # Create a geometry for the bounding box
    bbox_geom = [box(*bounding_box)]

    with rasterio.open(input_tiff) as src:
        # Crop the raster using the bounding box
        out_image, out_transform = mask(src, bbox_geom, crop=True)

        # Update metadata
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Write the subset to a temporary file
        with rasterio.open(temp_tiff, "w", **out_meta) as dest:
            dest.write(out_image)
            # Preserve band descriptions
            dest.descriptions = src.descriptions

    # Replace the original file
    os.replace(temp_tiff, input_tiff)

    print(f"Source GeoTIFF overwritten: {input_tiff}")

Source GeoTIFF overwritten: ./example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240813T082029Z_20240813T082329Z_epct_4c6b4cf7_FP.tif
Source GeoTIFF overwritten: ./example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240812T080756Z_20240812T081056Z_epct_abcd9ed3_FP.tif
Source GeoTIFF overwritten: ./example_data/varnavas_example/Satellite_Imagery/S3_OLCI/OLL1EFR_20240811T083407Z_20240811T083707Z_epct_89061cda_FP.tif


This ends our example on how to perform remote customisations on Data Store products using the Data Tailor. Feel free to adapt this script to search for your own products of interest, and customise with your chains as required. Guidance on chain customisation can be found on our <a href="https://user.eumetsat.int/resources/user-guides/data-store-detailed-guide#ID-Understanding-configuring-and-using-chains">Understanding, Configuring and Using Chains</a> page. If you need further help, you can contact us using the buttons at the bottom of the page.

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">GPL-3.0-or-later</a> license <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/atmosphere/trainings/nero-winter-school-2025">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>